In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.agents import AgentState


class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [ ]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage


@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(
        update={
            "favourite_colour": favourite_colour,
            "messages": [
                ToolMessage(
                    "Successfully updated favourite colour",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
        }
    )

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState,
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}},
)

In [6]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='91b50586-a267-4e21-ba18-65a9c106d324'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 219, 'prompt_tokens': 141, 'total_tokens': 360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8Z5ld2ai12FBytzmoXdPoHQfK4ri', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c53e2-a7c9-7330-991f-08f298dd2d12-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_Ll8ph9iZs1NQsmVPKbszA4Rv', 'type': 'tool_call'}], invalid_

In [ ]:
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green",
    },
    {"configurable": {"thread_id": "10"}},
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='e1f75a35-b083-4e25-b2a8-bccac7cf40c0'),
              AIMessage(content='Hello! I’m here and ready to help. I don’t have feelings, but I’m functioning well. How can I assist you today? If you’re just saying hi, nice to chat—how’s your day going?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 375, 'prompt_tokens': 142, 'total_tokens': 517, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8Z5tBJj7kyHg1DZ5awjPBRUcbpBH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c53e2-c49b-7222-903c-4b5d2371eaf4-0', 

## Read state

In [ ]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"


agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState,
)

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}},
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='bb4eaa11-7594-4157-9b08-704f0b5c0520'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 162, 'total_tokens': 509, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8Z5ydkw7QRwj8y6ieCrCksOnjkLB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c53e2-d66e-7401-97b2-9927e9244a2b-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_CjBEOhRbwI3BX6IKAHaHlOpS', 'type': 'tool_call'}], invalid_

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}},
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='bb4eaa11-7594-4157-9b08-704f0b5c0520'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 162, 'total_tokens': 509, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8Z5ydkw7QRwj8y6ieCrCksOnjkLB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c53e2-d66e-7401-97b2-9927e9244a2b-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_CjBEOhRbwI3BX6IKAHaHlOpS', 'type': 'tool_call'}], invalid_